<a href="https://colab.research.google.com/github/QaziSaim/CASE-STUDIES/blob/main/FAISS_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%%capture
!pip install langchain_google_genai
!pip install langchain_huggingface
!pip install langchain_community
!pip install faiss-cpu

In [4]:
from google.colab import userdata
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API')

In [6]:
path = "/content/data.txt"

In [7]:
loader = TextLoader(path)

In [8]:
docs = loader.load()

In [9]:
docs

[Document(metadata={'source': '/content/data.txt'}, page_content='Refund Policy:\n\nCustomers may Trequest a refund within 30 days of purchase. The item must be unused and To request a refund, email support@example.com with your order ID.')]

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=30)

In [11]:
chunks = splitter.split_documents(docs)

In [12]:
chunks

[Document(metadata={'source': '/content/data.txt'}, page_content='Refund Policy:'),
 Document(metadata={'source': '/content/data.txt'}, page_content='Customers may Trequest a refund within 30 days of'),
 Document(metadata={'source': '/content/data.txt'}, page_content='a refund within 30 days of purchase. The item'),
 Document(metadata={'source': '/content/data.txt'}, page_content='30 days of purchase. The item must be unused and'),
 Document(metadata={'source': '/content/data.txt'}, page_content='The item must be unused and To request a refund,'),
 Document(metadata={'source': '/content/data.txt'}, page_content='and To request a refund, email'),
 Document(metadata={'source': '/content/data.txt'}, page_content='To request a refund, email support@example.com'),
 Document(metadata={'source': '/content/data.txt'}, page_content='email support@example.com with your order ID.')]

In [13]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
vector_store = FAISS.from_documents(chunks, embedding=embeddings)

In [17]:
retriver = vector_store.as_retriever()

In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', "You are helpful assistant that can answer questions about the provided text.\n\n {context}.\n\n If you don't know just say i dont know {context}"),
        (MessagesPlaceholder(variable_name='chat_history')),
        ('user',"{input}")
    ]
)

In [24]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [25]:
chain = prompt_template | llm

In [31]:
def get_response(input_text, chat_history):
  matching_docs = retriver.get_relevant_documents(input_text)
  context = '\n'.join([doc.page_content for doc in matching_docs])
  response = chain.invoke({'input':input_text, 'context':context, 'chat_history':chat_history})
  return response.content

In [32]:
get_response('What is a refund policy for your company',[])

AttributeError: 'VectorStoreRetriever' object has no attribute 'get_relevant_documents'